In [21]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib 
%matplotlib inline
import os

import time

from bs4 import BeautifulSoup as bs
import re

In [2]:
# load the data  - 6 ml data points, takes a while to read

train = pd.read_csv("train.csv")

In [3]:
train.head()

,Id,Title,Body,Tags
0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6034195 entries, 0 to 6034194
Data columns (total 4 columns):
Id       int64
Title    object
Body     object
Tags     object
dtypes: int64(1), object(3)
memory usage: 184.1+ MB


In [5]:
train.Tags.value_counts()

android                                                 49783
php                                                     30664
jquery                                                  28079
javascript                                              24995
javascript jquery                                       24955
java                                                    23751
c#                                                      22889
python                                                  16470
php mysql                                               15798
mysql                                                   15362
c++                                                     15328
html css                                                13296
iphone                                                  12552
asp.net                                                 11586
ruby-on-rails                                           10066
c# asp.net                                               9963
css     

In [6]:
# sampling data for faster processing
train_set = train.sample(n=75000, random_state=42)


In [7]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 779393 to 1862034
Data columns (total 4 columns):
Id       500000 non-null int64
Title    500000 non-null object
Body     500000 non-null object
Tags     499997 non-null object
dtypes: int64(1), object(3)
memory usage: 19.1+ MB


In [8]:
train_set.Tags.value_counts()

android                                                 4114
php                                                     2415
jquery                                                  2374
javascript                                              2110
c#                                                      2018
java                                                    2017
javascript jquery                                       2011
python                                                  1386
mysql                                                   1313
c++                                                     1295
php mysql                                               1287
html css                                                1128
iphone                                                  1043
asp.net                                                  914
ruby-on-rails                                            822
c# asp.net                                               808
java android            

In [9]:
train_set.head()

,Id,Title,Body,Tags
779393,779394,LINQ Left Join On Not Equal Rows,<p>Im trying to display rows which does not ex...,linq join inequality
4988336,4988337,SSL Cert Client Validation - How to?,<p>We're setting up a VIP using F5 load balanc...,shell unix ssl ssl-certificate client-certific...
5207700,5207701,How to get MGSplitViewController working in Mo...,<p>I downloaded the MGSplitViewController proj...,iphone xcode binding monotouch simulator
5989550,5989551,"Writing my first gtk program, what's the best ...","<p>I've been reading tutorials, but one thing ...",gtk glade
1847890,1847891,Wordpress 2.8 Widget API is suitable for Worpr...,"<p>I am a little confused, the Wordpress 2.8 W...",plugin-development wordpress-version


In [12]:
# peek into data
train_set.Title.iloc[1:4]
train_set.Body.iloc[1]

# From the value counts and the title & body, it looks like the texts are about industry and technology

'<p>We\'re setting up a VIP using F5 load balancer in Windows 2008 Server. My IT dept has setup this VPI URL <a href="https://mydomain.com" rel="nofollow">https://mydomain.com</a> &amp; I need to test below scenarios but not sure how to do it?</p>\n\n<pre><code>SSL termination should happen for all requests.\nShould reject/drop the request if client SSL does not match.\nForward the request to my application, if client SSL matches.\n</code></pre>\n\n<p>Badically, the certificate is configured on Load balancer in such a way that LB will reject the request if client SSL does not match &amp; it\'ll accept it only if client SSL matches. </p>\n\n<p>Can any one tell me how to test this using some shell script or other way?</p>\n\n<p>Thanks!</p>\n'

In [27]:
train_set.Title + " " + train_set.Body

779393     LINQ Left Join On Not Equal Rows <p>Im trying ...
4988336    SSL Cert Client Validation - How to? <p>We're ...
5207700    How to get MGSplitViewController working in Mo...
5989550    Writing my first gtk program, what's the best ...
1847890    Wordpress 2.8 Widget API is suitable for Worpr...
5535870    How to build url in smarty template? <p>In tem...
2191009    Specified cast is not valid #3 <p>I have this ...
2373202    Blackberry - application signing <p>From what ...
1322986    Replace null cells with 0 or blank <p>How can ...
4822896    java multithread programming memory consistenc...
1387621    Custom tableview in appcelerator titanium <p>I...
4989456    Asp.net WebApi supports json, xml, jsonp? <p>T...
1938688    Deployment of Tomcat7 WAR <p>I ran <code>dpkg ...
1700726    iOS: Repeat XML Help Me <p>I have a code probl...
2373500    CVS Checkout in Windows from Linux Samba CVSRO...
5556064    Google Maps API V3 map container element dupli...
4958259    Stop or termi

In [15]:
# Data preprocessing 

print("before dropping duplicates: {}".format(len(train_set)))

# drop duplicates
train_set = train_set.drop_duplicates(subset=None, keep='first', inplace=False)

print("after dropping duplicates: {}".format(len(train_set)))

before dropping duplicates: 500000
after dropping duplicates: 500000


In [102]:
# clean html tags

def get_post(series):
    try:
        #get title
#         test_title = dataframe.Title

        # get body
#         test_body = dataframe.Body

        #get post 
#         dataframe['post'] = test_title + " " + test_body

        #clean post
        series = series.apply(lambda x: bs(markup=x, features='html.parser').getText())
    #     post = soup.getText()

        # remove symbols with regex
        series = series.apply(lambda x: re.sub(r"[^a-zA-Z0-9]"," ", x))
        series = series.apply(lambda x: x.replace('\n', ' '))
        
    #     post = re.sub(r"[^a-zA-Z0-9]", " ", post)
    except (TypeError, IndexError):
        print('typeerror')

    return series


In [107]:
train_set['cleaned_body'] = get_post(train_set.Body)

'We re setting up a VIP using F5 load balancer in Windows 2008 Server  My IT dept has setup this VPI URL https   mydomain com   I need to test below scenarios but not sure how to do it  SSL termination should happen for all requests  Should reject drop the request if client SSL does not match  Forward the request to my application  if client SSL matches   Badically  the certificate is configured on Load balancer in such a way that LB will reject the request if client SSL does not match   it ll accept it only if client SSL matches   Can any one tell me how to test this using some shell script or other way  Thanks  '

In [108]:
train_set.cleaned_body.head()

779393     Im trying to display rows which does not exist...
4988336    We re setting up a VIP using F5 load balancer ...
5207700    I downloaded the MGSplitViewController project...
5989550    I ve been reading tutorials  but one thing I m...
1847890    I am a little confused  the Wordpress 2 8 Widg...
Name: cleaned_body, dtype: object

In [110]:
train_set.Tags.value_counts()

android                                                 4114
php                                                     2415
jquery                                                  2374
javascript                                              2110
c#                                                      2018
java                                                    2017
javascript jquery                                       2011
python                                                  1386
mysql                                                   1313
c++                                                     1295
php mysql                                               1287
html css                                                1128
iphone                                                  1043
asp.net                                                  914
ruby-on-rails                                            822
c# asp.net                                               808
java android            